# Import Dependencies

In [2]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

# try play env

In [3]:
environment_name = 'Breakout-v0'

In [3]:
env = gym.make(environment_name)

In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action=action)
        score += reward
    print('Episodes={}, Score={}'.format(episode, score))
env.close()

Episodes=1, Score=1.0
Episodes=2, Score=1.0
Episodes=3, Score=2.0
Episodes=4, Score=0.0
Episodes=5, Score=1.0
Episodes=6, Score=0.0
Episodes=7, Score=0.0
Episodes=8, Score=1.0
Episodes=9, Score=1.0
Episodes=10, Score=1.0


In [10]:
env.action_space.sample()

3

In [11]:
env.observation_space.sample()

array([[[219,  70, 160],
        [235, 254, 100],
        [ 80,  66, 219],
        ...,
        [ 91,  20, 121],
        [ 82,  62, 174],
        [217, 198, 241]],

       [[ 61, 137, 147],
        [125, 220, 222],
        [ 62, 248,  46],
        ...,
        [ 59, 231, 213],
        [ 73,  20, 218],
        [ 18, 105, 110]],

       [[ 76, 206, 179],
        [ 32, 183, 254],
        [161, 196, 126],
        ...,
        [ 39,  17, 239],
        [146,  66, 111],
        [206, 255, 153]],

       ...,

       [[ 97,  36, 142],
        [114,  53, 130],
        [189,  79, 213],
        ...,
        [  6,  26, 155],
        [ 24,  68, 151],
        [160,  91, 224]],

       [[ 20,   0,  30],
        [ 11, 160, 199],
        [101, 230,  29],
        ...,
        [217, 118, 110],
        [151,  69, 243],
        [246, 101, 130]],

       [[ 30, 110,  47],
        [179, 245,  69],
        [125,  23, 184],
        ...,
        [ 42, 122, 164],
        [ 77, 166,  33],
        [ 94,   3, 188]]

# Vectorise Env and Train Model

In [12]:
env = make_atari_env(environment_name, n_envs=4, seed=666)

In [13]:
env = VecFrameStack(env, n_stack=4)

In [14]:
log_path = os.path.join('training', 'logs')

In [7]:
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

NameError: name 'log_path' is not defined

In [16]:
model.learn(total_timesteps=400000)

Logging to training\logs\A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 274      |
|    ep_rew_mean        | 1.49     |
| time/                 |          |
|    fps                | 245      |
|    iterations         | 100      |
|    time_elapsed       | 8        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.0542   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0339   |
|    value_loss         | 0.0517   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 292      |
|    ep_rew_mean        | 1.78     |
| time/                 |          |
|    fps                | 252      |
|    iterations         | 200      |
|    time_elapsed       | 15       |
|    total_timesteps    | 4000     |
| train

# Save and Reload Model

In [3]:
a2c_path = os.path.join('training', 'saved_models', 'A2C_model')

In [18]:
model.save(a2c_path)

In [19]:
del model

In [4]:
env = make_atari_env('Breakout-v0', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [5]:
model = A2C.load(a2c_path, env)

Wrapping the env in a VecTransposeImage.


# Evaluate and Test

In [10]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

c:\Users\jason\anaconda3\envs\rl\lib\site-packages\gym\envs\atari\environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


(12.5, 3.3541019662496847)

In [9]:
obs = env.reset()
dones = False
while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    print(rewards)
    env.render()